In [ ]:
from astropy.io import fits, ascii
from astropy.coordinates import SkyCoord, FK5
from astropy.wcs import WCS
from matplotlib.patches import Rectangle
from scipy import interpolate
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
# Primary Author: Maria Drout

data_dir = os.getenv("DATADIR") 
extinction_dir = data_dir + '4_MagellanicClouds/Extinction/ExtinctionCurve/'
filters_dir = data_dir + '2_Observations/5_Filters/'
models_dir = data_dir + '1_Models/Drout23/'

## Read in the Gordon Extinction Curves:

In [58]:
lmc_gordon_file = extinction_dir + 'lmc_avg_ext.dat'
smc_gordon_file = extinction_dir + 'smc_bar_avg.dat'
smc24_file = extinction_dir + 'SMC_Bumps.csv'

lmc_gordon = pd.read_csv(lmc_gordon_file,skiprows=7,names=['x','AlamAv','unc'],delimiter='\\s+')
lmc_Ang = (1./lmc_gordon['x'])*1e4
lmc_wave = np.flip(lmc_Ang)
lmc_AlamAv = np.flip(lmc_gordon['AlamAv'])

smc_gordon = pd.read_csv(smc_gordon_file,skiprows=7,names=['x','AlamAv','unc'],delimiter='\\s+')
smc_Ang = (1./smc_gordon['x'])*1e4
smc_wave = np.flip(smc_Ang)
smc_AlamAv = np.flip(smc_gordon['AlamAv'])

smc24 = pd.read_csv(smc24_file)
smc24_wave = smc24['wavelength']
smc24_AlamAv = smc24['extinction']


## Read in the Swift UVOT Filter Curves:

In [22]:
uvw2_filter_file = filters_dir + 'Swift_UVOT.UVW2.dat'
uvm2_filter_file = filters_dir + 'Swift_UVOT.UVM2.dat'
uvw1_filter_file = filters_dir + 'Swift_UVOT.UVW1.dat'

U_filter_file = filters_dir + 'Generic_Bessell.U.dat'
B_filter_file = filters_dir + 'Generic_Bessell.B.dat'
V_filter_file = filters_dir + 'Generic_Bessell.V.dat'
I_filter_file = filters_dir + 'Generic_Bessell.I.dat'

uvw2_filter = ascii.read(uvw2_filter_file,names=['wave','transmission'])
uvm2_filter = ascii.read(uvm2_filter_file,names=['wave','transmission'])
uvw1_filter = ascii.read(uvw1_filter_file,names=['wave','transmission'])

U_filter = ascii.read(U_filter_file,names=['wave','transmission'])
B_filter = ascii.read(B_filter_file,names=['wave','transmission'])
V_filter = ascii.read(V_filter_file,names=['wave','transmission'])
I_filter = ascii.read(I_filter_file,names=['wave','transmission'])

## Read in the Stripped Star Spectra:

In [37]:
#Note that the units of this are in erg/s/cm^2/Ang. And Ylva put them at 1kpc.
#spec1_file = 'Data/SpectralModels/M1_13.45_z006_SED.txt'
#spec1 = ascii.read(spec1_file,names=['wave','flux'],comment=';')
#spec1_file = models_dir + 'MS/f20/spectrum_M13.45_T30389.0_logg4.25_f20.txt'
spec1_file = models_dir + 'SHS/spectrum_Mstrip4.75404013.txt'
spec1 = ascii.read(spec1_file,names=['wave','flux','norm','mag'],comment='#')

#Restrict this a bit:
spec1_index = np.where((spec1['wave'] > 1400.) & (spec1['wave'] < 10000.))[0]
spec1b = spec1[spec1_index]


#spec2_file = 'Data/SpectralModels/M1_9.0_z006_SED.txt'
#spec2 = ascii.read(spec2_file,names=['wave','flux'],comment=';')
#spec2_file = models_dir + 'MS/f20/spectrum_M9.0_T25284.0_logg4.28_f20.txt'
spec2_file = models_dir + 'SHS/spectrum_Mstrip2.69005197.txt'
spec2 = ascii.read(spec2_file,names=['wave','flux','norm','mag'],comment='#')

#Restrict this a bit:
spec2_index = np.where((spec2['wave'] > 1400.) & (spec2['wave'] < 10000.))[0]
spec2b = spec2[spec2_index]

## Define Synthetic Photometry Functions


In [27]:
def DeRed(wave, flux, Av, wave_gordon, AlamAv):
    """Will de-redden an input spectrum based on the Gordon's Extinction Curve:

    Parameters:
    wave (angstrom): wavelengths of input spectrum in Angstrom
    flux (erg/s/cm/Ang): flux of spectrum; scaled version of this flux are fine.
    Av: Amount to de-redden by
    wave_gordon (Angstrom): wavelengths associated with the Gordon Curve
    AlamAv: associated array of Alambda/Av values for Gordon curve.

    Returns: a new flux array in same units as input
    """

    # Interpolate Gordone to the wavelengths of Spectrum:
    AlamAv_interp = np.interp(wave,wave_gordon,AlamAv)
    A_lambda = Av * AlamAv_interp

    #############
    NewSpec = flux * 10.0 ** (0.4 * A_lambda)

    return NewSpec


def SyntheticPhotomQuick(wave, flux, filter_wave,filter_transmission):
    """Calculates quick synthetic photom on an input spectrum. Quick = not properly zeropointed. 
    So this is mostly useful for calculated how much a spectral change would impact a magnitude (e.g. reddening coefficients)

    Parameters:
    wave (angstrom): wavelengths of input spectrum in Angstrom
    flux (): flux of spectrum
    filter wave: wavelength of filter.
    filter transmission: transmission/response of the filter

    """

    # Interpolate spectrum and response to save wavelengths:
    response2 = np.interp(wave, filter_wave, filter_transmission, left=0.0, right=0.0)
    
    flux_int = np.trapz(flux * wave * response2, x=wave) / np.trapz(response2*wave, x=wave)
    
    mag = -2.5 * np.log10(flux_int)
    
    return mag


def A_lam(wave, flux, wave_filt, transmission_filt,  Av, wave_gordon, AlamAv):
    
    mag1 = SyntheticPhotomQuick(wave, flux, wave_filt, transmission_filt)
    
    spec2_flux = DeRed(wave, flux, Av, wave_gordon, AlamAv)
    
    mag2 = SyntheticPhotomQuick(wave, spec2_flux, wave_filt, transmission_filt)
    
    A_lam = (mag1 - mag2)
    Coefficient = A_lam/Av
    
    return A_lam,Coefficient

## Now Actually Try this: 

In [34]:
#UVW2: 
Av_list = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8])

print('LMC UVW2')
for Av in Av_list:
    Alam_w2_lmc,coeff_w2_lmc = A_lam(spec1b['wave'],spec1b['flux'],uvw2_filter['wave'],uvw2_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_lmc,'Coeff ',coeff_w2_lmc)

print('SMC UVW2')
for Av in Av_list:
    Alam_w2_smc,coeff_w2_smc = A_lam(spec1b['wave'],spec1b['flux'],uvw2_filter['wave'],uvw2_filter['transmission'],Av,smc_wave,smc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_smc,'Coeff ',coeff_w2_smc)
    
    
print('LMC UVM2')
for Av in Av_list:
    Alam_w2_lmc,coeff_w2_lmc = A_lam(spec1b['wave'],spec1b['flux'],uvm2_filter['wave'],uvm2_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_lmc,'Coeff ',coeff_w2_lmc)

print('SMC UVM2')
for Av in Av_list:
    Alam_w2_smc,coeff_w2_smc = A_lam(spec1b['wave'],spec1b['flux'],uvm2_filter['wave'],uvm2_filter['transmission'],Av,smc_wave,smc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_smc,'Coeff ',coeff_w2_smc)
    
    
print('LMC UVW1')
for Av in Av_list:
    Alam_w2_lmc,coeff_w2_lmc = A_lam(spec1b['wave'],spec1b['flux'],uvw1_filter['wave'],uvw1_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_lmc,'Coeff ',coeff_w2_lmc)

print('SMC UVW1')
for Av in Av_list:
    Alam_w2_smc,coeff_w2_smc = A_lam(spec1b['wave'],spec1b['flux'],uvw1_filter['wave'],uvw1_filter['transmission'],Av,smc_wave,smc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_smc,'Coeff ',coeff_w2_smc)

LMC UVW2
Av: 0.1 Alam  0.2641782254241427 Coeff  2.641782254241427
Av: 0.2 Alam  0.5287486276984232 Coeff  2.643743138492116
Av: 0.3 Alam  0.7937032503536479 Coeff  2.6456775011788265
Av: 0.4 Alam  1.0590347628537309 Coeff  2.647586907134327
Av: 0.5 Alam  1.3247364208996864 Coeff  2.6494728417993727
Av: 0.6 Alam  1.5908020284283957 Coeff  2.6513367140473263
Av: 0.7 Alam  1.8572259012826997 Coeff  2.6531798589752853
Av: 0.8 Alam  2.124002832520354 Coeff  2.6550035406504424
SMC UVW2
Av: 0.1 Alam  0.35033095367315426 Coeff  3.5033095367315426
Av: 0.2 Alam  0.7025196640620841 Coeff  3.5125983203104205
Av: 0.3 Alam  1.0564980881670003 Coeff  3.521660293890001
Av: 0.4 Alam  1.4122017260724498 Coeff  3.5305043151811244
Av: 0.5 Alam  1.7695694908488768 Coeff  3.5391389816977536
Av: 0.6 Alam  2.128543559271389 Coeff  3.5475725987856483
Av: 0.7 Alam  2.489069209382265 Coeff  3.555813156260379
Av: 0.8 Alam  2.851094650045983 Coeff  3.563868312557479
LMC UVM2
Av: 0.1 Alam  0.2717657911297877 Coeff

In [38]:
#UVW2: 
Av_list = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8])

print('LMC UVW2')
for Av in Av_list:
    Alam_w2_lmc,coeff_w2_lmc = A_lam(spec2b['wave'],spec2b['flux'],uvw2_filter['wave'],uvw2_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_lmc,'Coeff ',coeff_w2_lmc)

print('SMC UVW2')
for Av in Av_list:
    Alam_w2_smc,coeff_w2_smc = A_lam(spec2b['wave'],spec2b['flux'],uvw2_filter['wave'],uvw2_filter['transmission'],Av,smc_wave,smc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_smc,'Coeff ',coeff_w2_smc)
    
    
print('LMC UVM2')
for Av in Av_list:
    Alam_w2_lmc,coeff_w2_lmc = A_lam(spec2b['wave'],spec2b['flux'],uvm2_filter['wave'],uvm2_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_lmc,'Coeff ',coeff_w2_lmc)

print('SMC UVM2')
for Av in Av_list:
    Alam_w2_smc,coeff_w2_smc = A_lam(spec2b['wave'],spec2b['flux'],uvm2_filter['wave'],uvm2_filter['transmission'],Av,smc_wave,smc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_smc,'Coeff ',coeff_w2_smc)
    
    
print('LMC UVW1')
for Av in Av_list:
    Alam_w2_lmc,coeff_w2_lmc = A_lam(spec2b['wave'],spec2b['flux'],uvw1_filter['wave'],uvw1_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_lmc,'Coeff ',coeff_w2_lmc)

print('SMC UVW1')
for Av in Av_list:
    Alam_w2_smc,coeff_w2_smc = A_lam(spec2b['wave'],spec2b['flux'],uvw1_filter['wave'],uvw1_filter['transmission'],Av,smc_wave,smc_AlamAv)
    print('Av:',Av,'Alam ',Alam_w2_smc,'Coeff ',coeff_w2_smc)

LMC UVW2
Av: 0.1 Alam  0.2641331206981583 Coeff  2.641331206981583
Av: 0.2 Alam  0.528668056752565 Coeff  2.643340283762825
Av: 0.3 Alam  0.7935963877658132 Coeff  2.645321292552711
Av: 0.4 Alam  1.0589103415358885 Coeff  2.647275853839721
Av: 0.5 Alam  1.3246027533300797 Coeff  2.6492055066601594
Av: 0.6 Alam  1.5906670268526923 Coeff  2.651111711421154
Av: 0.7 Alam  1.8570970968923355 Coeff  2.6529958527033366
Av: 0.8 Alam  2.1238873936235514 Coeff  2.6548592420294392
SMC UVW2
Av: 0.1 Alam  0.34966730978509375 Coeff  3.4966730978509375
Av: 0.2 Alam  0.7012171869720873 Coeff  3.5060859348604367
Av: 0.3 Alam  1.0545804078957808 Coeff  3.5152680263192693
Av: 0.4 Alam  1.4096913130335302 Coeff  3.5242282825838256
Av: 0.5 Alam  1.766487686851132 Coeff  3.532975373702264
Av: 0.6 Alam  2.1249106162049713 Coeff  3.5415176936749524
Av: 0.7 Alam  2.484904333594656 Coeff  3.549863333706652
Av: 0.8 Alam  2.846416050680066 Coeff  3.5580200633500825
LMC UVM2
Av: 0.1 Alam  0.2715704111551389 Coeff 

### Summarize the Results:

In [59]:
#LMC: Use Av=0.3 to calculate. But as seen above, not too bad. 
Av=0.3
Alam_w2_lmc,coeff_w2_lmc = A_lam(spec1b['wave'],spec1b['flux'],uvw2_filter['wave'],uvw2_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
Alam_m2_lmc,coeff_m2_lmc = A_lam(spec1b['wave'],spec1b['flux'],uvm2_filter['wave'],uvm2_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
Alam_w1_lmc,coeff_w1_lmc = A_lam(spec1b['wave'],spec1b['flux'],uvw1_filter['wave'],uvw1_filter['transmission'],Av,lmc_wave,lmc_AlamAv)

Alam_U_lmc,coeff_U_lmc = A_lam(spec1b['wave'],spec1b['flux'],U_filter['wave'],U_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
Alam_B_lmc,coeff_B_lmc = A_lam(spec1b['wave'],spec1b['flux'],B_filter['wave'],B_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
Alam_V_lmc,coeff_V_lmc = A_lam(spec1b['wave'],spec1b['flux'],V_filter['wave'],V_filter['transmission'],Av,lmc_wave,lmc_AlamAv)
Alam_I_lmc,coeff_I_lmc = A_lam(spec1b['wave'],spec1b['flux'],I_filter['wave'],I_filter['transmission'],Av,lmc_wave,lmc_AlamAv)

print('w2_coeff_lmc =',coeff_w2_lmc)
print('m2_coeff_lmc =',coeff_m2_lmc)
print('w1_coeff_lmc =',coeff_w1_lmc)
print('U_coeff_lmc =',coeff_U_lmc)
print('B_coeff_lmc =',coeff_B_lmc)
print('V_coeff_lmc =',coeff_V_lmc)
print('I_coeff_lmc =',coeff_I_lmc)

print('--------------------------')

#SMC: Use Av=0.2 to calcualte. 
Av=0.2
Alam_w2_smc,coeff_w2_smc = A_lam(spec1b['wave'],spec1b['flux'],uvw2_filter['wave'],uvw2_filter['transmission'],Av,smc_wave,smc_AlamAv)
Alam_m2_smc,coeff_m2_smc = A_lam(spec1b['wave'],spec1b['flux'],uvm2_filter['wave'],uvm2_filter['transmission'],Av,smc_wave,smc_AlamAv)
Alam_w1_smc,coeff_w1_smc = A_lam(spec1b['wave'],spec1b['flux'],uvw1_filter['wave'],uvw1_filter['transmission'],Av,smc_wave,smc_AlamAv)

Alam_U_smc,coeff_U_smc = A_lam(spec1b['wave'],spec1b['flux'],U_filter['wave'],U_filter['transmission'],Av,smc_wave,smc_AlamAv)
Alam_B_smc,coeff_B_smc = A_lam(spec1b['wave'],spec1b['flux'],B_filter['wave'],B_filter['transmission'],Av,smc_wave,smc_AlamAv)
Alam_V_smc,coeff_V_smc = A_lam(spec1b['wave'],spec1b['flux'],V_filter['wave'],V_filter['transmission'],Av,smc_wave,smc_AlamAv)
Alam_I_smc,coeff_I_smc = A_lam(spec1b['wave'],spec1b['flux'],I_filter['wave'],I_filter['transmission'],Av,smc_wave,smc_AlamAv)

print('w2_coeff_smc =',coeff_w2_smc)
print('m2_coeff_smc =',coeff_m2_smc)
print('w1_coeff_smc =',coeff_w1_smc)
print('U_coeff_smc =',coeff_U_smc)
print('B_coeff_smc =',coeff_B_smc)
print('V_coeff_smc =',coeff_V_smc)
print('I_coeff_smc =',coeff_I_smc)

print('--------------------------')
# SMC Dust Bump
Av=0.2
Alam_w2_smc,coeff_w2_smc = A_lam(spec1b['wave'],spec1b['flux'],uvw2_filter['wave'],uvw2_filter['transmission'],Av,smc24_wave,smc24_AlamAv)
Alam_m2_smc,coeff_m2_smc = A_lam(spec1b['wave'],spec1b['flux'],uvm2_filter['wave'],uvm2_filter['transmission'],Av,smc24_wave,smc24_AlamAv)
Alam_w1_smc,coeff_w1_smc = A_lam(spec1b['wave'],spec1b['flux'],uvw1_filter['wave'],uvw1_filter['transmission'],Av,smc24_wave,smc24_AlamAv)

Alam_U_smc,coeff_U_smc = A_lam(spec1b['wave'],spec1b['flux'],U_filter['wave'],U_filter['transmission'],Av,smc24_wave,smc24_AlamAv)
Alam_B_smc,coeff_B_smc = A_lam(spec1b['wave'],spec1b['flux'],B_filter['wave'],B_filter['transmission'],Av,smc24_wave,smc24_AlamAv)
Alam_V_smc,coeff_V_smc = A_lam(spec1b['wave'],spec1b['flux'],V_filter['wave'],V_filter['transmission'],Av,smc24_wave,smc24_AlamAv)
Alam_I_smc,coeff_I_smc = A_lam(spec1b['wave'],spec1b['flux'],I_filter['wave'],I_filter['transmission'],Av,smc24_wave,smc24_AlamAv)

print('SMC Dust Bump Coefficients:')
print('w2_coeff_smc =',coeff_w2_smc)
print('m2_coeff_smc =',coeff_m2_smc)
print('w1_coeff_smc =',coeff_w1_smc)
print('U_coeff_smc =',coeff_U_smc)
print('B_coeff_smc =',coeff_B_smc)
print('V_coeff_smc =',coeff_V_smc)
print('I_coeff_smc =',coeff_I_smc)

w2_coeff_lmc = 2.6456775011788265
m2_coeff_lmc = 2.7239186507872404
w1_coeff_lmc = 2.3542756089344983
U_coeff_lmc = 1.5630076992314084
B_coeff_lmc = 1.321009450105528
V_coeff_lmc = 1.033515610629993
I_coeff_lmc = 0.7369445126521157
--------------------------
w2_coeff_smc = 3.5125983203104205
m2_coeff_smc = 3.0628644693159046
w1_coeff_smc = 2.7406728513389034
U_coeff_smc = 1.7461226051185719
B_coeff_smc = 1.4126111680707432
V_coeff_smc = 1.035649240435248
I_coeff_smc = 0.5914748062195763
--------------------------
SMC Dust Bump Coefficients:
w2_coeff_smc = 3.365496431486452
m2_coeff_smc = 3.1499010949401196
w1_coeff_smc = 2.7728912897274327
U_coeff_smc = 1.7647815026759162
B_coeff_smc = 1.4203133681205138
V_coeff_smc = 1.051381971072196
I_coeff_smc = 0.7493007914506578
